In [ ]:
import os
import re
import sys
import cairosvg
import itertools
import numpy as np
import multiprocessing
from scipy.optimize import minimize
from scipy.sparse import csr_matrix
from scipy.stats import unitary_group
from typing import List, Union, Tuple
from utils import su2_encoding, CDTYPE
from IPython.display import display_svg
from mindquantum.simulator import Simulator
from mindquantum.core.circuit import Circuit
from mindquantum.core.operators import Hamiltonian
from utils import su2_encoding, str_special, CDTYPE
from mindquantum.simulator.utils import GradOpsWrapper
from mindquantum.core.gates import X, Y, Z, RX, RY, RZ, UnivMathGate


def save_img(circ: Circuit, name: str):
    img = os.path.join(os.getcwd(), 'img/qudit_sym')
    path = os.path.join(img, name)
    svg, png = f'{path}.svg', f'{path}.png'
    circ.svg().to_file(svg)
    for f in sorted(os.listdir(img)):
        if name in f and 'png' in f:
            match = re.search('_(\d+)', f)
            n = int(match.group(1)) + 1 if match else 1
            png = f'{path}_{n}.png'
    cairosvg.svg2png(url=svg, write_to=png, dpi=400, scale=8)
    if os.path.exists(svg):
        os.remove(svg)


def circuit_add(circ: Circuit) -> str:
    circ_add = 'Circuit()'
    para_gates = ['RX', 'RY', 'RZ']
    pr_list = circ.ansatz_params_name
    for g in circ:
        obj = g.obj_qubits[0] if len(g.obj_qubits) == 1 else g.obj_qubits
        ctrl = g.ctrl_qubits[0] if len(g.ctrl_qubits) == 1 else g.ctrl_qubits
        if g.name in para_gates:
            if len(g.ctrl_qubits) == 0:
                pr = [p for p in pr_list if p.startswith(f'{g.name}{obj}')][0]
                circ_add += f' + {g.name}(\'{pr}\').on({obj})'
            else:
                pr = [p for p in pr_list if p.startswith(f'{g.name}{obj}_{ctrl}')][0]
                circ_add += f' + {g.name}(\'{pr}\').on({obj}, {ctrl})'
        else:
            circ_add += f' + {g.name}({obj})' if len(g.ctrl_qubits) == 0 else f' + {g.name}({obj}, {ctrl})'
    return circ_add


def circuit_list(circ: Circuit) -> List[str]:
    circ_list = []
    for g in circ:
        obj = ''.join([str(i) for i in g.obj_qubits])
        ctrl = ''.join([str(i) for i in g.ctrl_qubits])
        if len(g.ctrl_qubits) == 0:
            g_str = f'{g.name}{obj}'
        else:
            g_str = f'{g.name}{obj}_{ctrl}'
        circ_list.append(g_str)
    return circ_list


def add_gate(gate_str: str, pr: Union[float, str] = 1):
    gate_list = ['X', 'Y', 'Z', 'CX', 'CY', 'CZ', 'RX', 'RY', 'RZ', 'CXRYCX']
    match = re.search('([A-Za-z]+)(\d+)_*(\d+)*', gate_str)
    gate = match.group(1)
    obj = match.group(2)
    ctrl = match.group(3)
    obj = int(obj) if len(obj) == 1 else [int(i) for i in obj]
    ctrl = int(ctrl) if ctrl and len(ctrl) == 1 else [int(i) for i in ctrl] if ctrl else []
    circ = Circuit()
    if gate == 'X':
        circ += X(obj)
    elif gate == 'Y':
        circ += Y(obj)
    elif gate == 'Z':
        circ += Z(obj)
    elif gate == 'CX':
        circ += X(obj, ctrl)
    elif gate == 'CY':
        circ += Y(obj, ctrl)
    elif gate == 'CZ':
        circ += Z(obj, ctrl)
    elif gate == 'RX':
        circ += RX(pr).on(obj, ctrl)
    elif gate == 'RY':
        circ += RY(pr).on(obj, ctrl)
    elif gate == 'RZ':
        circ += RZ(pr).on(obj, ctrl)
    elif gate == 'CXRYCX':
        circ += X(ctrl, obj)
        circ += RY(pr).on(obj, ctrl)
        circ += X(ctrl, obj)
    else:
        raise ValueError(f'Wrong input gate {gate} is not in {gate_list}')
    return circ


def gate_ind(dim: int, ind: List[int], gate_list: List[str]) -> List[str]:
    global nq, obj, U, Up
    nq = dim - 1
    obj = list(range(nq))
    U = unitary_group.rvs(2)
    Ud = np.eye(dim, dtype=CDTYPE)
    Ud[np.ix_(ind, ind)] = U

    p = np.eye(2**(dim - 1)) - su2_encoding(np.eye(dim))
    Ue = su2_encoding(Ud)
    Up = Ue + p

    obj_perm = list(itertools.permutations(obj, 2))
    obj_comb = list(itertools.combinations(obj, 2))

    gate_site = []
    for g in gate_list:
        if g in ['X', 'Y', 'Z']:
            for s in obj:
                gate_site.append(f'{g}{s}')
        elif g in ['CX', 'CY', 'CZ']:
            for s in obj_perm:
                gate_site.append(f'{g}{s[0]}_{s[1]}')
        elif g in ['RX', 'RY', 'RZ']:
            for s in obj:
                gate_site.append(f'{g}{s}')
            for s in obj_perm:
                gate_site.append(f'{g}{s[0]}_{s[1]}')
        elif g == 'CXRYCX':
            for s in obj_comb:
                gate_site.append(f'{g}{s[0]}_{s[1]}')
        else:
            raise ValueError(f'Wrong input gate: {g}')
    gate_site = list(dict.fromkeys(gate_site))
    return gate_site


def X_last(gate_tuple: Tuple[str], is_X_last: bool) -> Tuple[str]:
    x_ind = [i for i, g in enumerate(gate_tuple) if g.startswith('X')]
    y_ind = [i for i, g in enumerate(gate_tuple) if not g.startswith('X')]
    if is_X_last and len(x_ind) > 0 and len(y_ind) > 0:
        if min(x_ind) > max(y_ind):
            return gate_tuple
    else:
        return gate_tuple


def training(gate: np.ndarray, ansatz: Circuit, ind: List[int], count):

    def optimization(init_params: np.ndarray, sim_grad: GradOpsWrapper, loss_list: List[float] = None):
        f, g = sim_grad(init_params)
        loss = 1 - np.real(f)[0][0]
        grad = -np.real(g)[0][0]
        if loss_list is not None:
            loss_list.append(loss)
        return loss, grad

    def callback(curr_params: np.ndarray, tol: float = 1e-8):
        f, _ = sim_grad(curr_params)
        loss = 1 - np.real(f)[0][0]
        if loss < tol:
            raise StopIteration

    circ = Circuit() + UnivMathGate('Up', gate).on(obj)
    psi_rand = np.random.rand(2**nq) + 1j * np.random.rand(2**nq)
    sim = Simulator('mqvector', nq)
    sim.set_qs(psi_rand)
    sim.apply_circuit(circ)
    psi = sim.get_qs()
    rho = np.outer(psi, psi.conj())
    Ham = Hamiltonian(csr_matrix(rho))

    p_name = ansatz.ansatz_params_name
    p_num = len(p_name)

    sim.reset()
    sim.set_qs(psi_rand)
    sim_grad = sim.get_expectation_with_grad(Ham, ansatz)
    init_params = np.random.uniform(-np.pi, np.pi, p_num)

    res = minimize(optimization, init_params, args=(sim_grad, []), \
        method='BFGS', jac=True, callback=callback, options={'gtol': 1e-12})
    pr_res = dict(zip(p_name, res.x))
    ansatz = ansatz.apply_value(pr_res)

    if p_num <= para_num and res.fun < 1e-8:
        count.value += 1
        num = count.value
        name = f'd{dim}_U{ind[0]}{ind[1]}_num{str(num)}'
        print(name, circuit_list(ansatz), pr_res)
        if saveimg:
            save_img(ansatz, name)
        if display:
            display_svg(ansatz.svg())


def circuit(gate_tuple: Tuple[str], ind: List[int], count):
    circ = Circuit()
    corr = Circuit()
    for i, g in enumerate(gate_tuple):
        corr += add_gate(g, f'{g}_{i}')
    circ += corr
    circ += UnivMathGate('U', U).on(obj[0], obj[1:])
    circ += corr.hermitian()
    if len(circ.ansatz_params_name) > 0:
        training(Up, circ, ind, count)
    else:
        if np.allclose(Up, circ.matrix()):
            count.value += 1
            num = count.value
            name = f'd{dim}_U{ind[0]}{ind[1]}_num{str(num)}'
            print(name, circuit_list(circ))
            if saveimg:
                save_img(circ, name)
            if display:
                display_svg(circ.svg())


def running(dim: int, ind: List[int], gate_list: List[str], gate_num: int, pool_size: int = 32):
    gate_sites = gate_ind(dim, ind, gate_list)
    count = multiprocessing.Manager().Value('i', 0)

    if pool_size == 0 or sys.platform.startswith('win'):
        for gate_tuple in itertools.product(gate_sites, repeat=gate_num):
            gate_tuple = X_last(gate_tuple, is_X_last)
            if gate_tuple is not None:
                circuit(gate_tuple, ind, count)
    else:
        pool = multiprocessing.Pool(pool_size)
        for gate_tuple in itertools.product(gate_sites, repeat=gate_num):
            gate_tuple = X_last(gate_tuple, is_X_last)
            if gate_tuple is not None:
                pool.apply_async(circuit, (gate_tuple, ind, count))
        pool.close()
        pool.join()


dim = 4
ind = [0, 1]
gate_num = 5
para_num = 1
saveimg = True
display = False
is_X_last = False
gate_list = ['X', 'CXRYCX']
gate_list = ['X', 'CX', 'RY']

In [ ]:
dim = 4
ind = [0, 1]
gate_num = 5
para_num = 3
gate_list = ['CX', 'RY', 'X']
running(dim, ind, gate_list, gate_num)

In [ ]:
dim = 4
ind = [0, 2]
gate_num = 5
para_num = 2
gate_list = ['CX', 'RY', 'X']
running(dim, ind, gate_list, gate_num)

d4_U02_num1 ['X1_0', 'RY0_1', 'X1', 'X2_1', 'RY1_0', 'U0_12', 'RY1_0', 'X2_1', 'X1', 'RY0_1', 'X1_0'] {'RY0_1_1': 1.5707855882413475, 'RY1_0_4': 1.910644604631283}
d4_U02_num2 ['X1_0', 'RY0_1', 'X2', 'X1_2', 'RY2_0', 'U0_12', 'RY2_0', 'X1_2', 'X2', 'RY0_1', 'X1_0'] {'RY0_1_1': -4.712387445078707, 'RY2_0_4': -1.9106314781138554}
d4_U02_num3 ['X1_0', 'RY0_2', 'X1', 'X2_1', 'RY1_0', 'U0_12', 'RY1_0', 'X2_1', 'X1', 'RY0_2', 'X1_0'] {'RY0_2_1': -4.712431100982197, 'RY1_0_4': -1.9105876330986378}
d4_U02_num4 ['X1_0', 'RY0_2', 'X2', 'X1_2', 'RY2_0', 'U0_12', 'RY2_0', 'X1_2', 'X2', 'RY0_2', 'X1_0'] {'RY0_2_1': -4.7123882090183065, 'RY2_0_4': -1.9106255099056662}
d4_U02_num5 ['X1_0', 'X1', 'RY0_2', 'X2_1', 'RY1_0', 'U0_12', 'RY1_0', 'X2_1', 'RY0_2', 'X1', 'X1_0'] {'RY0_2_2': 1.5706677418989299, 'RY1_0_4': 1.910810048476222}
d4_U02_num6 ['X1_0', 'X2', 'RY0_1', 'X1_2', 'RY2_0', 'U0_12', 'RY2_0', 'X1_2', 'RY0_1', 'X2', 'X1_0'] {'RY0_1_2': 1.5707344578948732, 'RY2_0_4': 1.910600258526329}
d4_U02_nu

In [ ]:
dim = 4
ind = [0, 3]
gate_num = 3
para_num = 1
gate_list = ['CX', 'RY', 'X']
running(dim, ind, gate_list, gate_num, pool_size=0)

d4_U03_num1 ['X1', 'X2_1', 'X1_0', 'U0_12', 'X1_0', 'X2_1', 'X1']
d4_U03_num2 ['X1', 'X2_1', 'RY1_0', 'U0_12', 'RY1_0', 'X2_1', 'X1'] {'RY1_0_2': 3.141582388350447}
d4_U03_num3 ['X1', 'RY2_1', 'X1_0', 'U0_12', 'X1_0', 'RY2_1', 'X1'] {'RY2_1_1': 3.141589670762625}
d4_U03_num4 ['X2', 'X1_2', 'X2_0', 'U0_12', 'X2_0', 'X1_2', 'X2']
d4_U03_num5 ['X2', 'X1_2', 'RY2_0', 'U0_12', 'RY2_0', 'X1_2', 'X2'] {'RY2_0_2': 3.1419609919679496}
d4_U03_num6 ['X2', 'RY1_2', 'X2_0', 'U0_12', 'X2_0', 'RY1_2', 'X2'] {'RY1_2_1': 3.141592091223626}


In [ ]:
dim = 4
ind = [1, 2]
gate_num = 5
para_num = 3
gate_list = ['CX', 'RY', 'X']
running(dim, ind, gate_list, gate_num)

In [ ]:
dim = 4
ind = [1, 3]
gate_num = 5
para_num = 3
gate_list = ['CX', 'RY', 'X']
running(dim, ind, gate_list, gate_num)

In [ ]:
dim = 4
ind = [2, 3]
gate_num = 5
para_num = 3
gate_list = ['CX', 'RY', 'X']
running(dim, ind, gate_list, gate_num)

In [ ]:
dim = 3
para_num = 1
gate_list = ['X', 'CX', 'RY']
running(dim, [0, 1], gate_list, gate_num=3, pool_size=0)
running(dim, [0, 2], gate_list, gate_num=2, pool_size=0)
running(dim, [1, 2], gate_list, gate_num=3, pool_size=0)

d3_U01_num1 ['X0_1', 'X1', 'RY1_0', 'U0_1', 'RY1_0', 'X1', 'X0_1'] {'RY1_0_2': 1.570807096118784}
d3_U01_num2 ['X0_1', 'RY1_0', 'X1', 'U0_1', 'X1', 'RY1_0', 'X0_1'] {'RY1_0_1': -1.570796738615748}
d3_U02_num1 ['X1', 'X1_0', 'U0_1', 'X1_0', 'X1']
d3_U02_num2 ['X1', 'RY1_0', 'U0_1', 'RY1_0', 'X1'] {'RY1_0_1': 3.14142015846579}
d3_U02_num3 ['X1_0', 'X1', 'U0_1', 'X1', 'X1_0']
d3_U02_num4 ['X1_0', 'RY1', 'U0_1', 'RY1', 'X1_0'] {'RY1_1': 3.141551389359758}
d3_U02_num5 ['RY1_0', 'X1', 'U0_1', 'X1', 'RY1_0'] {'RY1_0_0': -3.1415792150936643}
d3_U12_num1 ['X0_1', 'RY1_0', 'X0', 'U0_1', 'X0', 'RY1_0', 'X0_1'] {'RY1_0_1': 1.5708142496619624}
d3_U12_num2 ['X0_1', 'RY1_0', 'X0_1', 'U0_1', 'X0_1', 'RY1_0', 'X0_1'] {'RY1_0_1': 1.5708065868951047}
d3_U12_num3 ['X1_0', 'RY0_1', 'X1_0', 'U0_1', 'X1_0', 'RY0_1', 'X1_0'] {'RY0_1_1': -1.5710719012304115}
